# EDA on World University Rankings dataset

## Introduction

In [ ]:
import os
from google.colab import userdata

# Retrieve the manual inputs from Secrets
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

# Verify connection
!kaggle datasets list

In [ ]:
# Download the specific dataset
!kaggle datasets download -d mylesoneill/world-university-rankings

In [ ]:
# Unzip the downloaded file
!unzip -q world-university-rankings.zip

# List the files to verify you have all 5
!ls

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

cwur = pd.read_csv('cwurData.csv')
times = pd.read_csv('timesData.csv')
shanghai = pd.read_csv('shanghaiData.csv')
exp = pd.read_csv(
    "education_expenditure_supplementary_data.csv",
    engine="python",
    on_bad_lines="skip",     # skip malformed lines
    dtype=str                # read everything as text first
)

attain = pd.read_csv('educational_attainment_supplementary_data.csv')
sclcont = pd.read_csv('school_and_country_table.csv')

***Here we can see three types of dataset -***
* Ranking Datasets -> cwurData, shanghaiData, timesData
* Country level contextual Datasets -> education_expenditure_supplementary_data.csv, educational_attainment_supplementary_data.csv
* Mapping Dataset -> school_and_country_table.csv

In [ ]:
lst = [cwur, times, shanghai, exp, attain, sclcont]
labels = ['cwurData', 'timesData', 'shanghaiData', 'education_expenditure_supplementary_data', 'educational_attainment_supplementary_data', 'school_and_country_table']

for i in range(len(lst)):
    df = lst[i]
    print(f'{labels[i]} - No. of rows in = {df.shape[0]} and columns = {df.shape[1]}')

***Detail of each column -***

**cwurData**
* world_rank -> World Rankings
* institution -> Name of institution
* country -> Country name
* national_rank -> Rank in the country of institution
* quality_of_education -> Rank of quality of education
* alumni_employment -> Rank for alumni employment
* quality_of_faculty -> Rank for quality of faculty
* publications -> Rank for publications
* influence -> Rank for influence
* citations -> No of students at university
* broad_impact -> rank for broad impact (only available for 2014 and 2015) (specific indicator used to measure a university's influence based on its research output)
* patents -> rank for patents
* score -> total score, used for determining world rank
* year -> year of ranking (2012 to 2015)

**shanghaiData**
* world_rank -> world rank for university. Contains rank ranges and equal ranks (eg. 101-152)
* university_name -> name of university
* national_rank -> rank of university within its country
* total_score -> total score, used to determine rank
* alumni -> Alumni Score, based on the number of alumni of an institution winning nobel prizes and fields medals
* award -> Award Score, based on the number of staff of an institution winning Nobel Prizes in Physics, Chemistry, Medicine
* hici -> HiCi Score, based on the number of Highly Cited Researchers selected by Thomson Reuters
* ns -> N&S Score, based on the number of papers published in Nature and Science
* pub -> PUB Score, based on total number of papers indexed in the Science Citation Index-Expanded and Social Science
* pcp -> PCP Score, the weighted scores of the above five indicators divided by the number of full time academic staff
* year -> year of ranking (2005 to 2015)

**timesData**

* world_rank -> world rank for the university. Contains rank ranges and equal ranks (eg. =94 and 201-250)
* university_name -> name of university
* country -> country of each university
* teaching -> university score for teaching (the learning environment)
* international -> university score international outlook (staff, students, research)
* research -> university score for research (volume, income and reputation)
* citations -> university score for citations (research influence)
* income -> university score for industry income (knowledge transfer)
* total_score -> total score for university, used to determine rank
* num_students -> number of students at the university
* student_staff_ratio -> Number of students divided by number of staff
* international_students -> Percentage of students who are international
* female_male_ratio -> Female student to Male student ratio
* year -> year of the ranking (2011 to 2016 included)

**education_expenditure**

* country -> A set of OECD countries, plus Brazil and the Russian Federation, plus an OECD average.
* institute_type -> All Institutes (including preprimary education and subsidies to households, not separately shown)
* direct_expenditure_type -> Public direct expenditure, private direct expenditure, or total (public + private) direct expenditure. (Private and total data are only available for 2011)
* Other(Year columns) -> It represents expenditure as a percentage of gross domestic product.

**educational_attainment**

* country_name -> Name of country
* series_name -> Description of the data for which measurement is there.
* Other(Year columns) -> Average data

**school_and_country**

* school_name -> Name of school
* country -> Country name

## EDA on Ranking datasets

### cwur Dataset

#### Data cleaning

**1. Missing values**

In [ ]:
cwur.head()

In [ ]:
cwur.describe()

In [ ]:
cwur.info()

In [ ]:
cwur['year'].value_counts()

In [ ]:
cwur['institution'].nunique()

In [ ]:
print(cwur.isna().sum())

In [ ]:
cwur.groupby(['institution'])['broad_impact'].median()

In [ ]:
cwur['broad_impact'] = cwur.groupby('institution')['broad_impact'].transform(lambda x: x.fillna(x.median()))

In [ ]:
print(cwur.isna().sum())

In [ ]:
cwur[cwur['broad_impact'].isna()]

In [ ]:
cwur[cwur['institution'] == 'Williams College']

It seems for Wiliams college only one entry is there and for that also broad impact is null. So filling the null value with median of the broad impact for the country.

In [ ]:
cwur['broad_impact'] = cwur.groupby('country')['broad_impact'].transform(lambda x: x.fillna(x.median()))

In [ ]:
print(cwur.isna().sum())

**2. Duplicates**

In [ ]:
cwur[cwur.duplicated(subset=['institution', 'year'])]

This shows that no duplicates are there with same institution name and year.

3. Wrong years

In [ ]:
cwur['year'].value_counts()

As we have only four years data, so year column is having correct values.

4. Columns with impossible values (e.g., negative scores)

In [ ]:
cwur.columns

In [ ]:
cwur.describe()

In [ ]:
cwur['world_rank'].nunique()

In [ ]:
cwur['world_rank'].shape

In [ ]:
duplicate_combinations = cwur.groupby(['world_rank', 'year']).size()
duplicate_combinations[duplicate_combinations > 1]

Data in all columns look right.

#### Data Exploration

***Types of columns*** ->

Score columns -> score

Ranking columns -> world_rank, national_rank, quality_of_education, publications, influence, broad_impact, patents

Numeric columns -> citations

Time based columns -> year

categorival columns -> institution, country

**1. Distribution (rank distribution, score distribution)**

In [ ]:
cwur[['world_rank', 'year']].value_counts()

In [ ]:
cwur.describe()

So world ranks are single numbers with every year the ranks being reset and maximum rank vaue in dataset is 1000.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
bins = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

for i in range(2):
  for j in range(2):
    year = 2012+2*i+j
    ax = axs[i, j]
    sns.histplot(data=cwur[cwur['year'] == year], x='world_rank', bins=bins, ax=ax)
    ax.set_title(f'World Rank Distribution - {year}')
    ax.set_xlabel('World Rank Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

This shows tha world ranks are evenly distributed and there is an incerease in unniversities ranked through the years.

In [ ]:
cwur_2012 = cwur[(cwur['year'] == 2012) & (cwur['world_rank'] >= 1) & (cwur['world_rank'] <= 100)].loc[:, ['institution', 'country']]
cwur_2012_counts = cwur_2012['country'].value_counts().reset_index()

cwur_2013 = cwur[(cwur['year'] == 2013) & (cwur['world_rank'] >= 1) & (cwur['world_rank'] <= 100)].loc[:, ['institution', 'country']]
cwur_2013_counts = cwur_2013['country'].value_counts().reset_index()

cwur_2014 = cwur[(cwur['year'] == 2014) & (cwur['world_rank'] >= 1) & (cwur['world_rank'] <= 100)].loc[:, ['institution', 'country']]
cwur_2014_counts = cwur_2014['country'].value_counts().reset_index()

cwur_2015 = cwur[(cwur['year'] == 2015) & (cwur['world_rank'] >= 1) & (cwur['world_rank'] <= 100)].loc[:, ['institution', 'country']]
cwur_2015_counts = cwur_2015['country'].value_counts().reset_index()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
data_list = [cwur_2012_counts, cwur_2013_counts, cwur_2014_counts, cwur_2015_counts]
for i in range(2):
  for j in range(2):
    ax = axs[i, j]
    sns.barplot(data=data_list[2*i+j], x='country', y='count', ax=ax)
    ax.set_title(f'World Top Rank Distribution - {year}')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_xlabel('Country')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
cwur_2012[cwur_2012['institution'].isin(cwur_2013['institution'])]

In [ ]:
cwur_2013[cwur_2013['institution'].isin(cwur_2014['institution'])]

In [ ]:
cwur_2014[cwur_2014['institution'].isin(cwur_2015['institution'])]

In [ ]:
cwur_2012[(cwur_2012['institution'].isin(cwur_2013['institution'])) & (cwur_2012['institution'].isin(cwur_2014['institution'])) & (cwur_2012['institution'].isin(cwur_2015['institution']))]

Around 75–80% of Top-100 universities remain the same year-to-year, indicating high stability.

Elite US and UK universities dominate the stable core of the Top-100.

Most new Top-100 entries are from China and East Asia.

Some of the universities like Harvard, MIT, Stanford and others have been consistently in top 100 ranks across the years.

In [ ]:
cwur.columns

In [ ]:
cwur_2012_qoe = cwur[(cwur['year'] == 2012) & (cwur['quality_of_education'] >= 1) & (cwur['quality_of_education'] <= 100)].loc[:, ['institution', 'country']]
cwur_2012_qoe_counts = cwur_2012_qoe['country'].value_counts().reset_index()

cwur_2013_qoe = cwur[(cwur['year'] == 2013) & (cwur['quality_of_education'] >= 1) & (cwur['quality_of_education'] <= 100)].loc[:, ['institution', 'country']]
cwur_2013_qoe_counts = cwur_2013_qoe['country'].value_counts().reset_index()

cwur_2014_qoe = cwur[(cwur['year'] == 2014) & (cwur['quality_of_education'] >= 1) & (cwur['quality_of_education'] <= 100)].loc[:, ['institution', 'country']]
cwur_2014_qoe_counts = cwur_2014_qoe['country'].value_counts().reset_index()

cwur_2015_qoe = cwur[(cwur['year'] == 2015) & (cwur['quality_of_education'] >= 1) & (cwur['quality_of_education'] <= 100)].loc[:, ['institution', 'country']]
cwur_2015_qoe_counts = cwur_2015_qoe['country'].value_counts().reset_index()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
data_list = [cwur_2012_qoe_counts, cwur_2012_qoe_counts, cwur_2012_qoe_counts, cwur_2012_qoe_counts]
for i in range(2):
  for j in range(2):
    ax = axs[i, j]
    sns.barplot(data=data_list[2*i+j], x='country', y='count', ax=ax)
    ax.set_title(f'World Top Rank Distribution by quality of education- {year}')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_xlabel('Country')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
cwur_2015_qoe

In [ ]:
plt.figure(figsize=(12, 5))
sns.barplot(data=cwur['country'].value_counts().tail(20).reset_index(),
    x='country', y='count')
plt.xticks(rotation=90)
plt.show()

The above figure shows some of the countries in lowest count of universities in top 1000.

In [ ]:
corr_matrix = cwur.corr(numeric_only=True)
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
plt.show()

The above figure shows the correlation between different ranks and we can see ranks to be heavily correlated.

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=cwur, x='score')
plt.show()

We can see that score is right skewed with very less universities having top scores and most universities having scores in 0-50 range.

In [ ]:
cwur['country'].value_counts(normalize = True).reset_index()

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(data = cwur['country'].value_counts().reset_index().head(10), x = 'country', y = 'count')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
sns.barplot(data = cwur['country'].value_counts().reset_index().tail(10), x = 'country', y = 'count')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(data = cwur['country'].value_counts(normalize = True).reset_index().head(10), x = 'country', y = 'proportion')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(data = cwur['country'].value_counts(normalize = True).reset_index().tail(10), x = 'country', y = 'proportion')
plt.tight_layout()
plt.show()

We can see high representation bias here with countries like USA, China, Japan, U.K. and Germany only accounting for almost 50% of universities in the dataset.

In [ ]:
cwur[cwur['year'] == 2015].groupby('country')['world_rank'].agg(['count', 'min', 'max', np.mean]).reset_index().sort_values(by = ['count', 'mean'], ascending = [False, True])

In [ ]:
india_universities = cwur[cwur['country'] == 'India']

fig, axs = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('University Scores in India (2012-2015)', fontsize=16)

years = [2012, 2013, 2014, 2015]

for i, year in enumerate(years):
    row = i // 2
    col = i % 2
    ax = axs[row, col]

    # Filter data for the current year and sort by score in descending order
    data_year = india_universities[india_universities['year'] == year].sort_values(by='score', ascending=False)

    if not data_year.empty:
        sns.barplot(data=data_year, x='institution', y='score', ax=ax, palette='viridis', hue='institution', legend=False)
        ax.set_title(f'{year}')
        ax.set_xlabel('Institution')
        ax.set_ylabel('Score')
        ax.tick_params(axis='x', rotation=90)
    else:
        ax.set_title(f'{year} - No Data Available')
        ax.set_xlabel('Institution')
        ax.set_ylabel('Score')

plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
plt.show()

In [ ]:
india_average_scores = cwur[cwur['country'] == 'India'].groupby('year')['score'].mean().reset_index()
print(india_average_scores)

In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(data=india_average_scores, x='year', y='score', palette='viridis', hue='year', legend=False)
plt.title('Average Score of Indian Universities (2014-2015)')
plt.xlabel('Year')
plt.ylabel('Average Score')
plt.show()

In [ ]:
india_rank_data = cwur[
    (cwur['country'] == 'India') &
    (cwur['year'].isin([2014, 2015]))
][['institution', 'world_rank']]

print(india_rank_data.head())

In [ ]:
plt.figure(figsize=(15, 7))

# Plot for 2014
plt.subplot(1, 2, 1) # 1 row, 2 columns, first plot
data_2014 = cwur[(cwur['country'] == 'India') & (cwur['year'] == 2014)].sort_values(by='world_rank', ascending=True)
sns.barplot(data=data_2014, x='institution', y='world_rank', palette='viridis', hue='institution', legend=False)
plt.title('World Ranks of Indian Universities (2014)')
plt.xlabel('Institution')
plt.ylabel('World Rank')
plt.xticks(rotation=90)

# Plot for 2015
plt.subplot(1, 2, 2) # 1 row, 2 columns, second plot
data_2015 = cwur[(cwur['country'] == 'India') & (cwur['year'] == 2015)].sort_values(by='world_rank', ascending=True)
sns.barplot(data=data_2015, x='institution', y='world_rank', palette='magma', hue='institution', legend=False)
plt.title('World Ranks of Indian Universities (2015)')
plt.xlabel('Institution')
plt.ylabel('World Rank')
plt.xticks(rotation=90)

plt.tight_layout()
plt.show()

### Times Dataset

#### Data cleaning

***Data transformation***

In [ ]:
times.head(5)

In [ ]:
times.info()

In [ ]:
times['world_rank'].value_counts()

In [ ]:
times['international'].value_counts()

In [ ]:
times['income'].value_counts()

In [ ]:
times['total_score'].value_counts()

In [ ]:
missing_percentage = times.isna().sum() / len(times) * 100
print(missing_percentage)

The `times` dataset contains several columns that are currently of `object` dtype but represent numerical values. These need to be converted to numerical types for proper analysis. Additionally, some columns contain special characters like commas, percentage signs, hyphens, and rank ranges that need to be handled during conversion.

Here's the plan to clean these columns:

1.  **`world_rank`**: This column contains strings like '1', '=94', '201-250'. I will create two new columns with starting range and end range. I will remove '=' from columns having '='.
2.  **`international`**: Convert to numeric after handling any non-numeric entries (e.g., '-').
3.  **`income`**: This column contains '-' for missing values. These will be treated as NaN (Not a Number) during conversion.
4.  **`total_score`**: Similar to `income`, this column might contain '-' or other non-numeric characters. These will be handled during conversion.
5.  **`num_students`**: This column contains commas (e.g., '20,152'). I will remove commas and then convert to numeric.
6.  **`international_students`**: This column contains percentage signs (e.g., '25%'). I will remove the '%' sign and convert to float.
7.  **`female_male_ratio`**: This column contains ratios (e.g., '33 : 67'). I will try to convert it to a float (e.g., 33/67), and if that fails, will keep it as NaN. For further in-depth analysis of gender ratio, it might be better to split it into two separate columns (female_percentage, male_percentage), but for a general numeric conversion, a single float ratio is a start.

In [ ]:
# Convert 'world_rank' to numeric, handling ranges and non-numeric characters
def convert_rank_lower(rank):
    if isinstance(rank, str):
        rank = rank.replace('=', '') # Remove '=' character
        if '-' in rank:
            return int(rank.split('-')[0]) # Take the lower bound of the range
        else:
            return int(0)
def convert_rank_upper(rank):
    if isinstance(rank, str):
        rank = rank.replace('=', '') # Remove '=' character
        if '-' in rank:
            return int(rank.split('-')[1]) # Take the lower bound of the range
        else:
            return int(200)
times['world_rank_transformed_lower'] = times['world_rank'].apply(convert_rank_lower)
times['world_rank_transformed_upper'] = times['world_rank'].apply(convert_rank_upper)

# Convert 'international', 'income', 'total_score' to numeric, coercing errors
times['international'] = pd.to_numeric(times['international'], errors='coerce')
times['income'] = pd.to_numeric(times['income'], errors='coerce')
times['total_score'] = pd.to_numeric(times['total_score'], errors='coerce')

# Convert 'num_students' by removing commas and then to numeric
times['num_students'] = times['num_students'].str.replace(',', '', regex=False)
times['num_students'] = pd.to_numeric(times['num_students'], errors='coerce')

# Convert 'international_students' by removing '%' and then to numeric
times['international_students'] = times['international_students'].str.replace('%', '', regex=False)
times['international_students'] = pd.to_numeric(times['international_students'], errors='coerce')

# Convert 'female_male_ratio' to a single float ratio, coercing errors
def convert_female_male_ratio(ratio_str):
    if isinstance(ratio_str, str) and ':' in ratio_str:
        try:
            female, male = map(int, ratio_str.split(' : '))
            return female / (female + male)
        except ValueError:
            return np.nan
    return np.nan
times['female_male_ratio'] = times['female_male_ratio'].apply(convert_female_male_ratio)

# Verify the data types after conversion
times.info()

In [ ]:
times.describe()

In [ ]:
missing_percentage = times.isna().sum() / len(times) * 100
print(missing_percentage)

In [ ]:
for col in ['international', 'income', 'total_score', 'num_students', 'student_staff_ratio', 'international_students', 'female_male_ratio']:
    # Fill missing values grouped by university_name first
    times[col] = times.groupby('university_name')[col].transform(lambda x: x.fillna(x.median()))
    # Fill any remaining missing values grouped by country
    times[col] = times.groupby('country')[col].transform(lambda x: x.fillna(x.median()))
    # Fill any remaining missing values with the global median of the column
    times[col] = times[col].fillna(times[col].median())

# Verify that all missing values have been handled
missing_percentage = times.isna().sum() / len(times) * 100
print(missing_percentage)

In [ ]:
times.head()

***Duplicates check***

In [ ]:
times.duplicated(subset=['university_name', 'year']).sum()

This means there are no duplicate universities in a specific year.

***Validity checks for values***

In [ ]:
times['year'].value_counts()

#### Data Exploration

***Column types identfication***

In [ ]:
times.columns

***

* Ranking columns -> 'world_rank'
* Score columns -> 'teaching', 'international',
       'research', 'citations', 'total_score', 'income'
* Numeric columns -> 'num_students'(number),
       'student_staff_ratio'(ratio), 'international_students'(%), 'female_male_ratio'(ratio),
* Time based columns -> 'year'

In [ ]:
years = times['year'].value_counts().sort_index().reset_index()
sns.barplot(data=years, x='year', y='count', hue='year', palette='viridis', legend=False)
plt.title('Number of Universities by Year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

So the data contains universities from 2011 - 2016 with an increase in number of universities over the years.

***Exploration***

In [ ]:
list(times['world_rank'].value_counts().index)

In [ ]:
times.columns

In [ ]:
def modify_rank(row):
  if row['world_rank_transformed_lower'] >=0 and row['world_rank_transformed_upper'] <= 200:
    return '0-200'
  elif row['world_rank_transformed_lower'] >= 201 and row['world_rank_transformed_upper'] <= 250:
    return '201-250'
  elif row['world_rank_transformed_lower'] >= 251 and row['world_rank_transformed_upper'] <= 300:
    return '251-300'
  elif row['world_rank_transformed_lower'] >= 301 and row['world_rank_transformed_upper'] <= 350:
    return '301-350'
  elif row['world_rank_transformed_lower'] >= 351 and row['world_rank_transformed_upper'] <= 400:
    return '351-400'
  elif row['world_rank_transformed_lower'] >= 401 and row['world_rank_transformed_upper'] <= 500:
    return '401-500'
  elif row['world_rank_transformed_lower'] >= 501 and row['world_rank_transformed_upper'] <= 600:
    return '501-600'
  else:
    return '601-800'

times['rank_final'] = times.apply(modify_rank, axis = 1)

In [ ]:
times.head()

In [ ]:
times['rank_final'].value_counts()

In [ ]:
times['rank_final'].isna().sum()

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 8))
for i in range(3):
  for j in range(2):
    year = 2011+2*i+j
    ax = axs[i, j]
    sns.histplot(data=times[times['year'] == year], x='rank_final', ax=ax)
    ax.set_title(f'World Rank Distribution - {year}')
    ax.set_xlabel('World Rank Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
times.loc[(times['year'] == 2016) & (times['rank_final'] == '0-200'), 'university_name'].head(10)

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 8))
for i in range(3):
  for j in range(2):
    year = 2011+2*i+j
    ax = axs[i, j]
    sns.histplot(data=times[times['year'] == year], x='total_score', ax=ax, kde = True)
    ax.set_title(f'Score Distribution - {year}')
    ax.set_xlabel('Score Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

The above plot shows that most ranks are between 50-60 scores and data is very much skewed to the right showing very less number of universities have top scores.

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 8))
for i in range(3):
  for j in range(2):
    year = 2011+2*i+j
    ax = axs[i, j]
    sns.histplot(data=times[times['year'] == year], x='teaching', ax=ax, kde = True)
    ax.set_title(f'Score Distribution(Teaching) - {year}')
    ax.set_xlabel('Score Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

Above plot shows that the teaching scores are mostly between 20 - 40 for most of the universities showing vey less teaching scores.

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 8))
for i in range(3):
  for j in range(2):
    year = 2011+2*i+j
    ax = axs[i, j]
    sns.histplot(data=times[times['year'] == year], x='research', ax=ax, kde = True)
    ax.set_title(f'Score Distribution(Research) - {year}')
    ax.set_xlabel('Score Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

Above plot shows most research scores are between 20 - 40 with scores slightly decreasing over the years.

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 8))
for i in range(3):
  for j in range(2):
    year = 2011+2*i+j
    ax = axs[i, j]
    sns.histplot(data=times[times['year'] == year], x='income', ax=ax, kde = True)
    ax.set_title(f'Score Distribution(Income) - {year}')
    ax.set_xlabel('Score Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

Above plot shows income scores are mostly between 30 - 40.

In [ ]:
times['country'].value_counts().reset_index()

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 12))
for i in range(3):
  for j in range(2):
    year = 2011+2*i+j
    ax = axs[i, j]
    sns.barplot(data=times[times['year'] == year]['country'].value_counts().reset_index().head(10), x='country', y = 'count', ax=ax)
    ax.set_title('No. of universities by country')
    ax.set_xlabel(f'Country - {year}')
    ax.set_ylabel('Count')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

plt.tight_layout()
plt.show()

We can see some countries are consistently having most number of universities.

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 12))
for i in range(3):
  for j in range(2):
    year = 2011+2*i+j
    ax = axs[i, j]
    sns.barplot(data=times[(times['year'] == year) & (times['rank_final'] == '0-200')]['country'].value_counts().reset_index().head(10), x='country', y = 'count', ax=ax)
    ax.set_title('No. of universities by country')
    ax.set_xlabel(f'Country - {year}')
    ax.set_ylabel('Count')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

plt.tight_layout()
plt.show()

Her we can see some changes in most number of universities by country in top 200 with China going out of the list and South Korea entering the list.

In [ ]:
times[times['year'] == 2016].groupby('country')['num_students'].mean().sort_values(ascending = False).head(10)

In [ ]:
times[times['year'] == 2016].groupby('country')['num_students'].mean().sort_values(ascending = False).tail(10)

In [ ]:
times_india = times[times['country'] == 'India']

In [ ]:
import textwrap

fig, axs = plt.subplots(3, 2, figsize=(18, 18))
fig.suptitle('Scores of universities in India over the years', fontsize=16)

years_to_plot = [2011, 2012, 2013, 2014, 2015, 2016]

for i in range(3):
    for j in range(2):
        year = years_to_plot[2*i+j]
        ax = axs[i, j]

        data_year = times_india[times_india['year'] == year]

        if not data_year.empty:
            data_year = data_year.sort_values(by='total_score', ascending=False)
            sns.barplot(data=data_year, x='university_name', y='total_score', ax=ax, palette='viridis', hue='university_name', legend=False)
            ax.set_title(f'{year}')
            ax.set_xlabel('Institution')
            ax.set_ylabel('Score')
            ax.set_xticks(ax.get_xticks())
            wrapped_labels = [textwrap.fill(label.get_text(), 25) for label in ax.get_xticklabels()] # Increased wrap_width to 25
            ax.set_xticklabels(wrapped_labels, rotation=45, ha='right')
        else:
            ax.set_title(f'{year} - No Data Available')
            ax.set_xlabel('Institution')
            ax.set_ylabel('Score')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

Institutes of India are mostly having score of 50 over the years.

### Shanghai Dataset

#### Data Cleaning

***Data Transformation***

In [ ]:
shanghai.head()

In [ ]:
shanghai.info()

In [ ]:
shanghai.describe()

In [ ]:
shanghai.isna().sum()

In [ ]:
list(shanghai['world_rank'].value_counts().index)

In [ ]:
def convert_rank_lower(rank):
    if isinstance(rank, str):
        if '-' in rank:
            return int(rank.split('-')[0])
        else:
            return int(0)
def convert_rank_upper(rank):
    if isinstance(rank, str):
        if '-' in rank:
            return int(rank.split('-')[1])
        else:
            return int(100)
shanghai['world_rank_transformed_lower'] = shanghai['world_rank'].apply(convert_rank_lower)
shanghai['world_rank_transformed_upper'] = shanghai['world_rank'].apply(convert_rank_upper)

shanghai.info()

In [ ]:
def modify_rank(row):
  if row['world_rank_transformed_lower'] >=0 and row['world_rank_transformed_upper'] <= 110:
    return '0-100'
  elif row['world_rank_transformed_lower'] >= 90 and row['world_rank_transformed_upper'] <= 210:
    return '101-200'
  elif row['world_rank_transformed_lower'] >= 190 and row['world_rank_transformed_upper'] <= 310:
    return '201-300'
  elif row['world_rank_transformed_lower'] >= 290 and row['world_rank_transformed_upper'] <= 410:
    return '301-400'
  elif row['world_rank_transformed_lower'] >= 390 and row['world_rank_transformed_upper'] <= 510:
    return '401-500'
  else:
    return '501-600'

shanghai['rank_final'] = shanghai.apply(modify_rank, axis = 1)

In [ ]:
shanghai.head()

In [ ]:
shanghai.info()

In [ ]:
shanghai.isna().sum()

In [ ]:
shanghai['national_rank'] = pd.to_numeric(shanghai['national_rank'], errors='coerce')

for col in ['total_score', 'national_rank', 'alumni', 'award', 'hici', 'ns', 'pub', 'pcp']:
    shanghai[col] = shanghai.groupby('university_name')[col].transform(lambda x: x.fillna(x.median()))
    shanghai[col] = shanghai[col].fillna(shanghai[col].median())

# Verify that all missing values have been handled
missing_percentage = shanghai.isna().sum() / len(shanghai) * 100
print(missing_percentage)

#### Data Exploration

In [ ]:
shanghai.columns

Rank columns -> world_rank, national_rank, rank_final

Score columns -> 'total_score',
       'alumni', 'award', 'hici', 'ns', 'pub', 'pcp'

Time based columns -> year


In [ ]:
years = shanghai['year'].value_counts().sort_index().reset_index()
sns.barplot(data=years, x='year', y='count', hue='year', palette='viridis', legend=False)
plt.title('Number of Universities by Year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

In [ ]:
shanghai[shanghai['year'] == 2015]['rank_final'].value_counts()

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 8))
for i in range(3):
  for j in range(2):
    year = 2010+2*i+j
    ax = axs[i, j]
    sns.histplot(data=shanghai[shanghai['year'] == year], x='rank_final', ax=ax)
    ax.set_title(f'World Rank Distribution - {year}')
    ax.set_xlabel('World Rank Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 8))
for i in range(3):
  for j in range(2):
    year = 2010+2*i+j
    ax = axs[i, j]
    sns.histplot(data=shanghai[shanghai['year'] == year], x='total_score', ax=ax, kde = True)
    ax.set_title(f'Score Distribution - {year}')
    ax.set_xlabel('Score Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

We can see that mostly total_score is between 20 and 30.

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 8))
for i in range(3):
  for j in range(2):
    year = 2010+2*i+j
    ax = axs[i, j]
    sns.histplot(data=shanghai[shanghai['year'] == year], x='alumni', ax=ax, kde = True)
    ax.set_title(f'Score Distribution - {year}')
    ax.set_xlabel('Score Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

Mostly alumni scores are between 0-40.

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 8))
for i in range(3):
  for j in range(2):
    year = 2010+2*i+j
    ax = axs[i, j]
    sns.histplot(data=shanghai[shanghai['year'] == year], x='award', ax=ax, kde = True)
    ax.set_title(f'Score Distribution - {year}')
    ax.set_xlabel('Score Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

Award scores are also mostly between 0 and 40.

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 8))
for i in range(3):
  for j in range(2):
    year = 2010+2*i+j
    ax = axs[i, j]
    sns.histplot(data=shanghai[shanghai['year'] == year], x='pcp', ax=ax, kde = True)
    ax.set_title(f'Score Distribution - {year}')
    ax.set_xlabel('Score Bins')
    ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

pcp score ranges between 0-40 having most concentration at score of 20.

In [ ]:
shanghai['rank_final'].value_counts()

In [ ]:
shanghai[(shanghai['year'] == 2015) & (shanghai['rank_final'] == '0-100')].head(10)

## EDA on supplementary data

### educational_attainment data

In [ ]:
attain.head()

In [ ]:
attain.info()

In [ ]:
attain[['series_name']].value_counts()

In [ ]:
attain.isna().sum()

In [ ]:
attain.columns

Missing years - 1988, 1989, 1994, 2014

In [ ]:
cwur['country'].unique()

In [ ]:
attain_countries = set(list(attain['country_name'].unique()))
cwur_countries = set(list(cwur['country'].unique()))
cwur_countries.difference(attain_countries)

The above countries are not in attain dataset.

#### Transforming year columns

Melting the year columns (1985-2015) in the `attain` DataFrame into two new columns: 'year' and 'value'. Then, display the first few rows of the transformed DataFrame and its information to verify the structure.

In [ ]:
year_cols = [col for col in attain.columns if col.isdigit()]
attain_melted = attain.melt(id_vars=['country_name', 'series_name'], value_vars=year_cols, var_name='year', value_name='value')

attain_melted['year'] = attain_melted['year'].astype(int)

print(attain_melted.head())
print(attain_melted.info())

In [ ]:
print(attain_melted['series_name'].value_counts())

#### Exploring Educational Attainment Trends

In [ ]:
attain_melted.head()

In [ ]:
attain_melted['series_name'].unique()

***Checking highest attainment countries.
For this we will add percentage population with tertiary education in all age groups using Barro-Lee dataset.***

In [ ]:
bins = ['15+', '15-19', '20-24', '25+', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75+']
lst_of_tert_edu_percent = [f'Barro-Lee: Percentage of population age {x} with tertiary schooling. Completed Tertiary' for x in bins]
lst_of_age_pop = [f'Barro-Lee: Population in thousands, age {x}, total' for x in bins]

df_tert_edu_percent = attain_melted[attain_melted['series_name'].isin(lst_of_tert_edu_percent)]
df_age_pop = attain_melted[attain_melted['series_name'].isin(lst_of_age_pop)]

In [ ]:
df_tert_edu_percent_new = df_tert_edu_percent.reset_index(drop = True).loc[:, ['country_name', 'year', 'value']]

In [ ]:
df_tert_edu_percent_new

In [ ]:
df_age_pop_new = df_age_pop.reset_index(drop = True).loc[:, 'value'] * 1000

In [ ]:
df_age_pop_new

In [ ]:
df_tert_merged = df_tert_edu_percent_new.merge(df_age_pop_new, how = 'inner', left_index = True, right_index = True)

In [ ]:
df_tert_merged['tert_pop'] = (df_tert_merged['value_x'] * df_tert_merged['value_y']) / 100

In [ ]:
df_tert_merged = df_tert_merged.dropna()

In [ ]:
df_tert_merged

In [ ]:
df_tert_merged = df_tert_merged.groupby(['country_name', 'year'])[['value_y', 'tert_pop']].sum().reset_index()

In [ ]:
df_tert_merged

In [ ]:
df_tert_merged['tert_percent'] = (df_tert_merged['tert_pop'] / df_tert_merged['value_y']) * 100

In [ ]:
df_tert_merged

In [ ]:
df_tert_merged[df_tert_merged['year'] == 2010].sort_values(by = 'tert_percent', ascending = False)

In [ ]:
selected_countries = ['United States', 'China', 'Japan', 'India', 'United Kingdom']
filtered_df_tert_merged = df_tert_merged[df_tert_merged['country_name'].isin(selected_countries)]

plt.figure(figsize=(12, 6))
sns.lineplot(
    data=filtered_df_tert_merged,
    x='year',
    y='tert_percent',
    hue='country_name',
    marker='o',
    palette='deep'
)
plt.title(f'Tertiary Education Trend')
plt.xlabel('Year')
plt.ylabel('Value')
plt.grid(True)
plt.legend(title='Country')
plt.tight_layout()
plt.show()

***Generating line plots to visualize the trends of selected educational attainment metrics over the years for a few representative countries to highlight how educational attainment has evolved historically.***

In [ ]:
selected_series_names = [
    'Barro-Lee: Average years of primary schooling, age 15+, total',
    'UIS: Percentage of population age 25+ with at least completed lower secondary education (ISCED 2 or higher). Total'
]

selected_countries = ['United States', 'China', 'Japan', 'India', 'United Kingdom']

filtered_attain = attain_melted[
    (attain_melted['series_name'].isin(selected_series_names)) &
    (attain_melted['country_name'].isin(selected_countries))
].dropna(subset=['value'])

print(filtered_attain.head())

In [ ]:
filtered_attain['series_name'].unique()

In [ ]:
unique_series = filtered_attain['series_name'].unique()
fig, axes = plt.subplots(1, 2, figsize = (12, 6))
i = 0

for series in unique_series:
    sns.lineplot(
        data=filtered_attain[filtered_attain['series_name'] == series],
        x='year',
        y='value',
        hue='country_name',
        marker='o',
        palette='deep',
        ax = axes[i]
    )
    i+=1
    plt.title(f'Educational Attainment Trend: {series}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.grid(True)
    plt.legend(title='Country')
plt.tight_layout()
plt.show()


***Analyzing Educational Attainment Distribution by Country***

Creating visualizations to compare different educational attainment metrics across various countries for the latest year available in the filtered_attain DataFrame.


In [ ]:
latest_year = filtered_attain['year'].max()
latest_year_data = filtered_attain[filtered_attain['year'] == latest_year]

unique_series_latest = latest_year_data['series_name'].unique()

for series in unique_series_latest:
    plt.figure(figsize=(12, 6))
    sns.barplot(
        data=latest_year_data[latest_year_data['series_name'] == series],
        x='country_name',
        y='value',
        palette='viridis',
        hue='country_name',
        legend=False
    )
    plt.title(f'Educational Attainment Distribution: {series} - {latest_year}')
    plt.xlabel('Country')
    plt.ylabel('Value')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

### education_expenditure data

In [ ]:
exp.head()

In [ ]:
exp.info()

In [ ]:
exp.isna().sum()

In [ ]:
exp.describe()

In [ ]:
exp.columns

In [ ]:
exp['country'].value_counts()

In [ ]:
exp['institute_type'].value_counts()

In [ ]:
exp[exp['country'] == 'Australia']

So, above analysis tells that there are 37 countries and for every country there are nine entries. For every country there are three types of institutes and for every country and institute type combination there are three entries for expenditure type making it total nine entries.

#### Transforming data

In [ ]:
year_cols = ['1995', '2000', '2005', '2009', '2010', '2011']

for col in year_cols:
    exp[col] = pd.to_numeric(exp[col], errors='coerce')

In [ ]:
exp.columns

In [ ]:
exp_melted = exp.melt(id_vars = ['country', 'institute_type', 'direct_expenditure_type'], value_vars = year_cols, var_name = 'year', value_name = 'value')
exp_melted

In [ ]:
exp_melted.isna().sum()

In [ ]:
exp_melted.dropna(inplace = True)

In [ ]:
exp_melted.isna().sum()

In [ ]:
exp_melted.head()

In [ ]:
exp_melted['country'].value_counts()

#### Highest and lowest spending

In [ ]:
exp_melted['country'] = exp_melted['country'].str.strip()
exp_melted['institute_type'] = exp_melted['institute_type'].str.strip()
exp_melted['direct_expenditure_type'] = exp_melted['direct_expenditure_type'].str.strip()
exp_melted['year'] = exp_melted['year'].str.strip()

In [ ]:
selected_countries = ['OECD Average', 'Australia', 'Japan', 'United States', 'United Kingdom', 'Russian Federation']

filtered_exp = exp_melted[
    (exp_melted['institute_type'] == 'All Institutions') & (exp_melted['year'] == '2011') & (exp_melted['direct_expenditure_type'] == 'Total')
]

In [ ]:
filtered_exp.sort_values(by = 'value', ascending = False).head(10)

In [ ]:
filtered_exp.sort_values(by = 'value', ascending = False).tail(10)

#### Visualizing expenditure over the years

In [ ]:
exp_melted.head()

In [ ]:
selected_countries = ['OECD Average', 'Australia', 'Japan', 'United States', 'United Kingdom', 'Russian Federation']

filtered_exp = exp_melted[
    (exp_melted['country'].isin(selected_countries)) & (exp_melted['institute_type'] == 'All Institutions')
]

print(filtered_exp.head())

In [ ]:
filtered_exp['direct_expenditure_type'].unique()

In [ ]:
unique_exp_type = filtered_exp['direct_expenditure_type'].unique()
fig, axes = plt.subplots(1, 3, figsize = (18, 6)) # Increased figure size for better readability
i=0
for typ in unique_exp_type:
    sns.lineplot(
        data=filtered_exp[filtered_exp['direct_expenditure_type'] == typ],
        x='year',
        y='value',
        hue='country',
        marker='o',
        palette='deep',
        ax=axes[i],

    )
    axes[i].set_title(f'{typ} Expenditure') # Add title to each subplot
    axes[i].legend(title='Country') # Add legend to each subplot
    i+=1

plt.tight_layout()
plt.show()

In [ ]:
selected_countries = ['OECD Average', 'Australia', 'Japan', 'United States', 'United Kingdom', 'Russian Federation']

filtered_exp1 = exp_melted[
    (exp_melted['country'].isin(selected_countries)) & (exp_melted['direct_expenditure_type'] == 'Public')
]

print(filtered_exp1.head())

In [ ]:
filtered_exp1['institute_type'].value_counts()

In [ ]:
unique_inst_type = filtered_exp1['institute_type'].unique()
fig, axes = plt.subplots(1, 3, figsize = (18, 6)) # Increased figure size for better readability
i=0
for typ in unique_inst_type:
    sns.lineplot(
        data=filtered_exp1[filtered_exp1['institute_type'] == typ],
        x='year',
        y='value',
        hue='country',
        marker='o',
        palette='deep',
        ax=axes[i],

    )
    axes[i].set_title(f'{typ} institute') # Add title to each subplot
    axes[i].legend(title='Country') # Add legend to each subplot
    i+=1

plt.tight_layout()
plt.show()

## EDA on school_and_country_table

In [ ]:
sclcont.head()

In [ ]:
sclcont.info()

In [ ]:
sclcont.describe()

In [ ]:
sclcont.isna().sum()

### Adding countries in Shanghai dataset from school_and_country_table

In [ ]:
shanghai.head()

Shanghai dataset doesn't have countries. We will try to map the uiversities in Shanghai dataset with countries from school_and_country_table.

In [ ]:
len(shanghai['university_name'].unique())

In [ ]:
sclcont_list = set(list(sclcont['school_name'].unique()))
shanghai_list = set(list(shanghai['university_name'].unique()))

In [ ]:
len(shanghai_list.difference(sclcont_list))

In [ ]:
shanghai.head()

In [ ]:
shanghai = shanghai.merge(sclcont[['school_name', 'country']], how = 'left', left_on = 'university_name', right_on = 'school_name')

In [ ]:
shanghai.drop(columns = ['school_name'], inplace = True)

In [ ]:
shanghai.head()

In [ ]:
shanghai.info()

In [ ]:
shanghai.isna().sum()

### Universities that appear in multiple datasets

In [ ]:
cwur.head(1)

In [ ]:
shanghai.head(1)

In [ ]:
times.head(1)

In [ ]:
cwur_list = set(list(cwur['institution'].unique()))

In [ ]:
len(cwur_list)

In [ ]:
len(cwur_list.difference(sclcont_list))

In [ ]:
times_list = set(list(times['university_name'].unique()))

In [ ]:
len(times_list)

In [ ]:
len(times_list.difference(sclcont_list))

In [ ]:
shanghai_list = set(list(shanghai['university_name'].unique()))

In [ ]:
len(shanghai_list)

In [ ]:
len(shanghai_list.difference(sclcont_list))

In [ ]:
common_uni = times[(times['university_name'].isin(shanghai['university_name'])) & ((times['university_name'].isin(cwur['institution'])))]['university_name'].unique()

In [ ]:
len(common_uni)

Therefore, 362 universities appear across all datasets.

## Cross Ranking Analysis

In [ ]:
def mod_rank(rank):
  if isinstance(rank, str):
        rank = rank.replace('=', '')
        if '-' in rank:
            return int(rank.split('-')[0])
        else:
            return int(rank)

In [ ]:
times['rank_modified'] = times['world_rank'].apply(mod_rank)

In [ ]:
times.head(1)

In [ ]:
times.info()

In [ ]:
times['rank_modified'].unique()

In [ ]:
shanghai['rank_modified'] = shanghai['world_rank'].apply(mod_rank)

In [ ]:
shanghai.head(1)

In [ ]:
shanghai.info()

In [ ]:
shanghai['rank_modified'].unique()

In [ ]:
cwur_common_ranked = cwur[cwur['institution'].isin(common_uni)].groupby('institution')['world_rank'].median().reset_index()

In [ ]:
times_common_ranked = times[times['university_name'].isin(common_uni)].groupby('university_name')['rank_modified'].median().reset_index()

In [ ]:
shanghai_common_ranked = shanghai[shanghai['university_name'].isin(common_uni)].groupby('university_name')['rank_modified'].median().reset_index()

In [ ]:
ranked_merged = cwur_common_ranked.merge(times_common_ranked, how = 'left', left_on = 'institution', right_on = 'university_name')

In [ ]:
ranked_merged.drop(columns=['institution'], inplace = True)

In [ ]:
ranked_merged.rename(columns={'world_rank': 'rank_cwur', 'rank_modified': 'rank_times'}, inplace=True)

In [ ]:
ranked_merged.head()

In [ ]:
ranked_merged = ranked_merged.merge(shanghai_common_ranked, how = 'left', on = 'university_name')

In [ ]:
ranked_merged.rename(columns={'rank_modified': 'rank_shanghai'}, inplace=True)

In [ ]:
ranked_merged.head()

In [ ]:
ranked_merged.info()

In [ ]:
ranked_merged['row_variance'] = ranked_merged.select_dtypes(include='number').var(axis=1)

In [ ]:
ranked_merged.sort_values(by = 'row_variance', ascending = False)

Some of the universities with high variance in ranks are -

'King Abdulaziz University, 'University of Greifswald', 'Murdoch University', 'University of Calcutta' etc.

In [ ]:
cwur['year'].unique()

In [ ]:
times['year'].unique()

In [ ]:
shanghai['year'].unique()

In [ ]:
len(cwur['country'].unique())

In [ ]:
len(shanghai['country'].unique())

In [ ]:
len(times['country'].unique())

In [ ]:
times['rank_modified'].describe()

In [ ]:
times['rank_modified_adjusted'] = (times['rank_modified'] / 600) * 500

In [ ]:
shanghai['rank_modified'].describe()

In [ ]:
shanghai['rank_modified_adjusted'] = (shanghai['rank_modified'] / 402) * 500

In [ ]:
cwur['world_rank'].describe()

In [ ]:
cwur['rank_modified_adjusted'] = (cwur['world_rank'] / 1000) * 500

In [ ]:
country_merged = times[times['year'] == 2015].groupby(['country'])['rank_modified_adjusted'].median().reset_index()

In [ ]:
country_merged = country_merged.merge(shanghai[shanghai['year'] == 2015].groupby(['country'])['rank_modified_adjusted'].median().reset_index(), on = 'country', how  ='left')

In [ ]:
country_merged = country_merged.merge(cwur[cwur['year'] == 2015].groupby(['country'])['rank_modified_adjusted'].median().reset_index(), on = 'country', how  ='left')

In [ ]:
country_merged

Some countries appearing only in one ranking - Macau , Morocco

In [ ]:
country_merged['rank_combined'] = country_merged.select_dtypes(include=['number']).mean(axis=1)

In [ ]:
country_merged

In [ ]:
country_merged.sort_values(by = 'rank_combined', ascending = True).head(10)

These are the top ten countries by rank across all ranking datasets.

In [ ]:
df_tert_merged[df_tert_merged['year'] == 2010]['country_name'].value_counts()

In [ ]:
df_tert_merged = df_tert_merged[df_tert_merged['year'] == 2010]

In [ ]:
df_tert_merged.head()

In [ ]:
exp_melted.head()

In [ ]:
exp_melted['year'].unique()

In [ ]:
exp_melted['institute_type'].unique()

In [ ]:
exp_melted['direct_expenditure_type'].unique()

In [ ]:
exp_melted = exp_melted[(exp_melted['year'] == '2011') & (exp_melted['direct_expenditure_type'] == 'Total') & (exp_melted['institute_type'] == 'All Institutions')]

In [ ]:
country_merged = country_merged.merge(df_tert_merged, how = 'left', left_on = 'country', right_on = 'country_name')

In [ ]:
country_merged.head()

In [ ]:
country_merged = country_merged.merge(exp_melted, how = 'left', left_on = 'country', right_on = 'country')

In [ ]:
country_merged.head()

In [ ]:
country_merged = country_merged[['country', 'rank_combined', 'tert_percent', 'value']]

In [ ]:
country_merged.sort_values(by = 'rank_combined', ascending = True)